In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, callback_context, no_update
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "MiiUnHy02i1L3"

# Connect to database via CRUD Module
db = AnimalShelter()
records = db.read({}) or []
df = pd.DataFrame.from_records(records)

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
# Only when id is present
if '_id' in df.columns:
    df.drop(columns = ['_id'], inplace = True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__, suppress_callback_exceptions = True)

image_filename = 'GS_Logo.png'
encoded_image = None
if os.path.exists(image_filename):
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()
image_element = html.Img(src=(f"data:image/png;base64,{encoded_image}" if encoded_image else ""),
                         style={'height': '80px'}, alt='Grazioso Salvare')


# Interactive filtering controls (dropdown and rescue)
filter_controls = html.Div([
    html.Div([
        html.Label("Filter by Animal Type:"),
        dcc.Dropdown(
            id='filter-type',
            options=(
                [{"label": t, "value": t} for t in sorted(df['animal_type'].dropna().unique())]
                if 'animal_type' in df.columns else
                [{"label": v, "value": v} for v in ["Dog", "Cat", "Bird", "Livestock", "Other"]]
            ),
            placeholder="Select an Animal Type",
            clearable=True,
            style={'width': '100%'}
        )
    ], style={'width':'40%','display':'inline-block','padding':'10px','verticalAlign':'top'}),

    html.Div([
        html.Label("Rescue Type"),
        dcc.RadioItems(
            id='filter-rescue',
            options=[
                {'label':'Water Rescue', 'value':'Water'},
                {'label':'Mountain or Wilderness Rescue', 'value':'Mountain'},
                {'label':'Disaster or Individual Tracking', 'value':'Disaster'},
                {'label':'None (no rescue filter)', 'value':'None'}
            ],
            value='None',
            labelStyle={'display': 'block'}
        ),
        html.Button("Reset Filters", id='reset-filters', n_clicks=0, style={'marginTop':'8px'})
    ], style={'width':'28%','display':'inline-block','padding':'10px','verticalAlign':'top'})
])

        
# Enhanced Data Table
initial_selected = [0] if len(df) > 0 else []
data_table = dash_table.DataTable(
    id='datatable-id',
    columns=[{"name": i, "id": i} for i in df.columns],
    data=df.to_dict('records'),
    page_size=10,
    style_table={'overflowX': 'auto'},
    style_cell={'textAlign': 'left', 'minWidth': '120px', 'width': '150px', 'maxWidth': '300px'},
    filter_action='native',
    sort_action='native',
    sort_mode='multi',
    row_selectable='single',
    selected_rows=initial_selected,
    page_action='native'
)


# App Layout
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(image_element, style={'textAlign': 'center', 'padding': '5px'}),
    html.Div(html.P("Created by: Dylan Harmon"), style={'textAlign':'center'}),
    html.Hr(),
    filter_controls,
    html.Hr(),
    data_table,
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])
    
###########################
# Simple Pie Chart helper
###########################

def pie_by_breed(df_local):
    if df_local is None or df_local.empty:
        return px.pie(names=["No data"], values=[1], title='Breed Distribution')
    if 'breed' not in df_local.columns:
        return px.pie(names=["Unknown"], values=[len(df_local)], title='Breed Distribution')
    agg = df_local.groupby('breed').size().reset_index(name='count').sort_values('count', ascending=False)
    fig = px.pie(agg, names='breed', values='count', title='Breed Distribution')
    fig.update_traces(textposition='inside', textinfo='percent+label')
    return fig
    


#############################################
# Interaction Between Components / Controller
#############################################

# Rescue Mapping
RESCUE_MAP = {
    'Water': {
        "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    'Mountain': {
        "breed": {"$in": ["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    'Disaster': {
        "breed": {"$in": ["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    }
}

# Data Table: update data according to dropdown and rescue
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value'),
               Input('filter-rescue', 'value')])
def update_dashboard(filter_type, filter_rescue):
    query = {}
    if filter_type:
        query["animal_type"] = filter_type

    if filter_rescue != 'None' and filter_rescue in RESCUE_MAP:
        query.update(RESCUE_MAP[filter_rescue])

    records = db.read(query) or []
    dff = pd.DataFrame.from_records(records)
    if not dff.empty and '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)
    return dff.to_dict('records')

# Reset filters callback
@app.callback(
    Output('filter-type', 'value'),
    Output('filter-rescue', 'value'),
    Input('reset-filters', 'n_clicks')
)
def reset_filters(n_clicks):
    if not n_clicks:
        return no_update, no_update
    return None, 'None'


##################################
# Build + Return pie chart and map
##################################

# Graph Callback (simple pie) — uses derived_virtual_data so chart follows table filters/sorting/searching
@app.callback(Output('graph-id', 'children'),
              [Input('datatable-id', 'derived_virtual_data')])
def update_graphs(viewData):
    dfl = pd.DataFrame.from_dict(viewData) if viewData else pd.DataFrame(columns=df.columns)
    fig = pie_by_breed(dfl)
    return [dcc.Graph(id='pie-id', figure=fig)]

# This callback will highlight selected columns
@app.callback(Output('datatable-id', 'style_data_conditional'),
              [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]

# Map update
@app.callback(Output('map-id', 'children'),
              [Input('datatable-id', 'derived_virtual_data'),
               Input('datatable-id', 'derived_virtual_selected_rows')])
def update_map(viewData, selected_rows):
    if not viewData:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    row = 0 if not selected_rows else selected_rows[0]
    if row < 0 or row >= len(dff):
        return []

    # prefer explicit lat/long fields if present
    lat = lon = None
    if 'location_lat' in dff.columns and 'location_long' in dff.columns:
        try:
            lat = float(dff.loc[row, 'location_lat'])
            lon = float(dff.loc[row, 'location_long'])
        except Exception:
            lat = lon = None
    else:
        # default positional indices (change if needed)
        lat_idx, lon_idx = 13, 14
        if dff.shape[1] > max(lat_idx, lon_idx):
            try:
                lat = float(dff.iat[row, lat_idx])
                lon = float(dff.iat[row, lon_idx])
            except Exception:
                lat = lon = None
        if lat is None or lon is None:
            # fallback: try to find plausible numeric columns
            for col in dff.columns:
                try:
                    val = float(dff.loc[row, col])
                except Exception:
                    continue
                if -90 <= val <= 90 and lat is None:
                    lat = val
                elif -180 <= val <= 180 and lon is None:
                    lon = val
                if lat is not None and lon is not None:
                    break

    if lat is None or lon is None:
        return [html.Div("No valid latitude/longitude available for the selected row.")]

    def safe_get_by_index(df_local, r, idx, default=""):
        if idx < df_local.shape[1]:
            try:
                return str(df_local.iat[r, idx])
            except Exception:
                return default
        return default

    breed = safe_get_by_index(dff, row, list(dff.columns).index('breed') if 'breed' in dff.columns else 4, "Unknown breed")
    name = safe_get_by_index(dff, row, list(dff.columns).index('name') if 'name' in dff.columns else 9, "Unknown name")
    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[lat, lon], children=[dl.Tooltip(breed), dl.Popup([html.H1("Animal Name"), html.P(name)])])
    ])]

# Run app
app.run_server()

Dash app running on https://sardineorange-fluidphilips-3000.codio.io/proxy/8050/
